In [1]:
%cd ..

C:\Users\wuyua\Projects\PycharmProjects\RxExperiments


In [2]:
import numpy as np
import pandas as pd
import rx
from rx import operators
from rx import scheduler
from rx import subject

import logging
import datetime

from configs.config import GlobalConfig
GlobalConfig.initialize_global_configuration("configs/defaults.json")

from utils.logging import configure_logger_to_output
logging_output = configure_logger_to_output(level=logging.INFO)

# Description
Collect some image and Raman, FBRM information for 35 degc saturation
Solubility = 0.143003 g/g
water = 50 g
need taurine = 7.150 g

## Shared initialization

In [3]:
from utils.mqtt_wrapper import MQTTClientWrapper
from sources.harvesters_source import HarvestersSource
from sinks.save_image_sink import SaveImageSink
from sinks.save_data_sink import SaveDataSink
from sinks.visualization_sink import JupyterImageSink, PlotlyVisualizationSink
from datamodel.image import Image
from operators import data_framer
from plotly import graph_objects as go
from controls.camera_controller import CameraControl, CameraControlCommand
from controls.fp50 import FP50Command, FP50Control
from controls.mqtt_pump import MQTTPump
from sources.raman_source import RamanSource
from sources.mqtt_ds18 import MQTTDS18Source
from utils.auto_disposable import AutoDisposable

import ipywidgets as widgets


In [4]:
# MQTT connection
client = MQTTClientWrapper("experiment_taurine")
client.connect("192.168.43.1")
client.loop_start()

### Image analysis

In [5]:
# Image acquisition
image_acquisiton_enabled = True

image_source = HarvestersSource().pipe(operators.filter(lambda x: image_acquisiton_enabled), operators.share())
image_save_sink = SaveImageSink()
camera_control = CameraControl()
current_exposure = 10

# save image logic
image_save_subscription = image_source.subscribe(image_save_sink)

# camera controller logic
image_mean_brightness_source = image_source.pipe(
    operators.map(lambda x: x.image.mean())
)

# enable fan and trigger at the beginning
camera_control.on_command(CameraControlCommand().set_exposure(current_exposure).set_trigger(True))

# dataframed source
image_brightness_branch = image_mean_brightness_source.pipe(
    operators.sample(2),
    operators.map(lambda x: pd.DataFrame(data=[{"value": x}], index=[datetime.datetime.now()]))
)

In [6]:
# Front end panels
# camera visuallization logic
camera_image_sink = JupyterImageSink(name="Camera")
camera_visuallization_subscription = image_source.pipe(operators.sample(1)).subscribe(camera_image_sink)

# camera brightness visuallization logic
mean_brightness_sink = PlotlyVisualizationSink(name="Mean brightness")
mean_brightness_visuallization_subscription = image_brightness_branch.pipe(
    data_framer.data_framer(),
    operators.map(lambda x: [go.Scatter(x=x.index, y=x.value)])
).subscribe(mean_brightness_sink)

# DS18

In [7]:
ds18 = MQTTDS18Source(client)
ds18_save = SaveDataSink("ds18", "value", auto_timestamp = True)
ds18_save_subs = AutoDisposable(ds18.pipe(
    operators.map(lambda x: pd.DataFrame(data=[{"value": x["value"]}])),
).subscribe(ds18_save))
ds18_plot = PlotlyVisualizationSink(name="Temperature")
ds18_plot_subs = AutoDisposable(ds18.pipe(
    operators.map(lambda x: pd.DataFrame(data=[{"value": x["value"]}])),
    data_framer.data_framer(window_length = 300, auto_timestamp = True),
    operators.map(lambda x: [go.Scatter(x=x.timestamp, y=x.value, name="Temperature")]),
).subscribe(ds18_plot))

## Water bath and temperature control

In [8]:
fp50 = FP50Control(client)
fp50_df = fp50.pipe(
    operators.map(lambda x: pd.DataFrame(data=[{"power": x.power, "setpoint": x.setpoint, "temperature": x.temperature}]))
)
fp50_save = SaveDataSink("fp50", "data", auto_timestamp = True)

fp50_save_subs = AutoDisposable(fp50_df.subscribe(fp50_save))
fp50_power_plot = PlotlyVisualizationSink(name="Power")
fp50_power_plot_subs = AutoDisposable(fp50_df.pipe(
    data_framer.data_framer(window_length = 300, auto_timestamp = True),
    operators.map(lambda x: [go.Scatter(x=x.timestamp, y=x.power, name="Power")]),
).subscribe(fp50_power_plot))

fp50_temperature_plot = PlotlyVisualizationSink(name="Temperature")
fp50_temperature_plot_subs = AutoDisposable(fp50_df.pipe(
    data_framer.data_framer(window_length = 300, auto_timestamp = True),
    operators.map(lambda x: [go.Scatter(x=x.timestamp, y=x.temperature, name="Temperature")]),
).subscribe(fp50_temperature_plot))

# Pump control

In [9]:
# back end
pump_control = MQTTPump(client, "pump", "6712580")

# Raman

In [10]:
# backend configuration
raman_source = RamanSource(client, "raman").pipe(operators.share())

raman_dataframe_source = raman_source.pipe(
    operators.map(
        lambda x: pd.DataFrame(
            index=[datetime.datetime.now()], 
            data=[dict(zip(x["wave_number"], x["data"]))],
        )
    )
)

# save file logic
raman_save_sink = SaveDataSink("raman", "data")
raman_save_subscription = raman_dataframe_source.subscribe(raman_save_sink)

In [11]:
# front end configuration
raman_visuallization = PlotlyVisualizationSink(name="Raman")
raman_visuallization_subscription = raman_source.pipe(
    operators.map(lambda x: [go.Scatter(x=x["wave_number"], y=x["data"], )])
).subscribe(raman_visuallization)

raman_visuallization.figure.update_layout(xaxis={"autorange": "reversed", "title":"$Wave number (cm^{-1})$"}, yaxis={"title": "Count"})

;

''

# Concentrations

In [12]:
from properties.taurine import TaurineProperty
taurine_property = TaurineProperty(r"C:\Users\wuyua\OneDrive - The University of Western Ontario\Research\Taurine\Raman Calibration\data\raman_pca_linear_model.pkl")
def func(x):
    d = list(x[0]["data"])
    d.append(x[1]["value"])
    input_data = np.array(d).reshape((1, -1))
    y = taurine_property.raman_concentration(input_data)
    solids = y[0][0]
    solute = y[0][1]
    solubility = taurine_property.solubility(x[1]["value"])
    return {"solids": solids, "solute":solute, "solubility":solubility}
        
conc_src = raman_source.pipe(
    operators.with_latest_from(ds18),
    operators.map(func),
    operators.share()
)

# front end configuration

conc_plot = PlotlyVisualizationSink(name="Concentration")
conc_save = SaveDataSink("conc", "value", auto_timestamp = True)
conc_save_subs = AutoDisposable(conc_src.pipe(
    operators.map(lambda x: pd.DataFrame(data=[x])),
).subscribe(conc_save))
conc_plot_subs = AutoDisposable(conc_src.pipe(
    operators.map(lambda x: pd.DataFrame(data=[x])),
    data_framer.data_framer(window_length = 300, auto_timestamp = True),
    operators.map(lambda x: [go.Scatter(x=x.timestamp, y=x.solute, name="solute"), go.Scatter(x=x.timestamp, y=x.solubility, name="solubility"), go.Scatter(x=x.timestamp, y=x.solids, name="solids")]),
).subscribe(conc_plot))

# FBRM

In [13]:
from sources.fbrm_source import FBRMSource

# backend configuration
fbrm_source = FBRMSource(client, "fbrm").pipe(operators.share())

fbrm_dataframe_source = fbrm_source.pipe(
    operators.map(
        lambda x: pd.DataFrame(
            index=[datetime.datetime.now()], 
            data=[dict(zip(x["sizes"], x["counts"]))],
        )
    )
)

# save file logic
fbrm_save_sink = SaveDataSink("fbrm", "data")
fbrm_save_subscription = fbrm_dataframe_source.subscribe(fbrm_save_sink)

In [14]:
# front end configuration
fbrm_visuallization = PlotlyVisualizationSink(name="FBRM")
fbrm_visuallization_subscription = fbrm_source.pipe(
    operators.map(lambda x: [go.Scatter(x=x["sizes"], y=x["counts"], )])
).subscribe(fbrm_visuallization)

fbrm_visuallization.figure.update_layout(xaxis={"type":"log", "title":"$Size (\mu m)$"}, yaxis={"title": "Count"})

fbrm_count_visuallization = PlotlyVisualizationSink(name="FBRM statistics")
fbrm_count_visuallization_subscription = fbrm_source.pipe(
    operators.map(lambda x: pd.DataFrame(index=[datetime.datetime.now()], data=[{
        "total": np.sum(x["counts"])
    }])),
    data_framer.data_framer(),
    operators.map(lambda x: [go.Scatter(x=x.index, y=x.total, )])
).subscribe(fbrm_count_visuallization)
;

''

## Organize widgets

In [15]:
from ipywidgets import Layout, Box, VBox
# logging panel
# logging_output.layout = Layout(border="solid", width="100%", height="200px", overflow="scroll")

# image panel
image_panel_layout = Layout(display="flex", flex_flow="row", border="solid", width="100%")
mean_brightness_figure = Box(children=[mean_brightness_sink.fig], layout=Layout(flex="1 1 0%", width="auto"))
camera_image_sink.figure.layout = Layout(flex="1 1 0%", width="50%")
image_panel = Box(children=[mean_brightness_sink.figure, camera_image_sink.figure], layout=image_panel_layout)

# temperature panel
temperature_panel = VBox(children=[fp50_power_plot.figure, fp50_temperature_plot.figure, ds18_plot.figure], layout=
    Layout(border="solid", width="100%", display="flex", flex_flow="column")
)

# FBRM panel
fbrm_panel = Box(children=[fbrm_visuallization.figure], layout=
    Layout(border="solid", width="100%", display="flex", flex_flow="column")
)


# Raman Panel
raman_panel = Box(children=[raman_visuallization.figure], layout=
    Layout(border="solid", width="100%", display="flex", flex_flow="column")
)

# Concentration panel
conc_panel = Box(children=[conc_plot.figure], layout=
    Layout(border="solid", width="100%", display="flex", flex_flow="column")
)


# control panel
event_logger_text = widgets.Text(
    description='Event annotation:',
    layout=Layout(width="100%"),
)
event_logger_text.on_submit(lambda x: logging.getLogger("annotation").info(x.value))


control_panel = Box(children=[event_logger_text], layout=Layout(border="solid", width="100%"))

# wrap up
panel = widgets.VBox([image_panel, temperature_panel, fbrm_panel, raman_panel, conc_panel, control_panel])

display(panel)

    'data': [{'type': 'scatter',
              'uid': 'be40f328-31…

In [19]:
image_acquisiton_enabled = False

In [17]:
camera_control.on_command(CameraControlCommand().set_exposure(11.0))

In [22]:
delta_t = 5
start =20
cooling_rate = -4/3600 # d/s
end = 40
def enable_fcn(x):
    if x < 35 and not image_acquisiton_enabled:
        image_acquisiton_enabled = True
    return x
subs = rx.interval(delta_t).pipe(
    operators.map(lambda x: start - x*delta_t * cooling_rate),
    operators.take_while(lambda x: x < end),
    operators.map(lambda x: FP50Command(x))
).subscribe(fp50)

In [23]:
subs.dispose()

In [20]:
fp50.on_command(FP50Command(20.))

In [21]:
pump_control.on_command(0)

Event record:

* 10:34 Clean Raman and FBRM.
* 10:39 Cooling to 31 degc.
* 10:42 Clean Raman
* 10:52 delete all images, start cooling, 0.1 deg/min, RPM = 400
* 11:12 FBRM nucleation
* 11:13 Image sees nucleation

Strange, many large crystals before massive nucleation. Why?
* 11:19 clean raman
* 11:30 clean raman
* 11:40 Raman is strange, change RPM to 200 to reduce vortex
* 12:09 Reached 24 degc. Stay there and see if the supersaturation will be consumed
* 12:21 to 19

30-20 with 2% seed load (20-45um)
30-20 solids out = 1.889 g. Therefore, add seed = 0.03778
* 12:33 Heat to 40 to dissolve everything
* 12:41 Clean FBRM probe
* 12:47 Cool to 30.5
* 12:58 Cool to 30, then add solids (prevent dissolution)
* 1:03 add seeds, wait 10min


### Make it 35 according to gao, 
since already add 0.03778 seed and 6.0304 g solute, now add extra 1.14907g to make it saturated at 35.

Amount of seed 2% final product

p.solubility(35)*50 - p.solubility(25)*50 == 2.2050314032909553. 

Therefore, add 0.044g

* 1:15 Add 1.1489g
* 1:21 FBRM reading ready cool to 35
* 1:29 Linear (0.4 d/m) cool to 33  <- too fast!!
* 1:34 add 0.0442g seeds @ internal 34degc
* 1:37 leave seeds for 5 min
* 1:44 T = 34 to break agglomeration a little bit, hold steady for 5 min  <- next time do stay at 34, 
* 1:52 0.1d/min to 25. 
* 2:07 Seems the aspect ratio is very high! (even before cooling start) To verify this, since add seeds only introduce 0.1 degc solubility change, just do another cooling without seed


## Try no seed as said before
* 3:43 heat to 40
* 3:59 cool to 36